# Chapter 8 - Recurrent Neural Networks

**Recurrent neural networks** (RNNs) are deep learning models that capture the dynamics of sequences via *recurrent connections*, which can be thought of as cycles in the network of nodes. 

Recurrent neural networks are *unrolled* across time steps (or sequence steps), with the *same underlying parameters* applied at each step.

While the standard connections are applied *synchronously* to propagate each layer’s activations to the subsequent layer *at the same time step*, the recurrent connections are *dynamic*, passing information across adjacent time steps.

![](../imgs/ch09/unfolded-rnn.svg)

RNNs can be thought of as feedforward neural networks where each layer's parameters (both conventional and recurrent) are shared across time steps.

## 9.1. Working with Sequences